In [1]:
import sqlite3

conn = sqlite3.connect('libros.db')
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS categorias (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre TEXT NOT NULL UNIQUE,
    url TEXT NOT NULL
)
''')
print("Tabla categorias creada")


cursor.execute('''
CREATE TABLE IF NOT EXISTS libros (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    titulo TEXT NOT NULL,
    precio REAL NOT NULL,
    rating INTEGER CHECK(rating >= 0 AND rating <= 5),
    stock INTEGER DEFAULT 0,
    categoria_id INTEGER NOT NULL,
    FOREIGN KEY (categoria_id) REFERENCES categorias(id)
)
''')
print("Tabla libros creada")


cursor.execute('''
CREATE TABLE IF NOT EXISTS autores (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre TEXT NOT NULL UNIQUE
)
''')
print("Tabla autores creada")


cursor.execute('''
CREATE TABLE IF NOT EXISTS libros_autores (
    libro_id INTEGER NOT NULL,
    autor_id INTEGER NOT NULL,
    PRIMARY KEY (libro_id, autor_id),
    FOREIGN KEY (libro_id) REFERENCES libros(id),
    FOREIGN KEY (autor_id) REFERENCES autores(id)
)
''')
print("Tabla libros_autores creada")


conn.commit()
conn.close()

Tabla categorias creada
Tabla libros creada
Tabla autores creada
Tabla libros_autores creada


In [2]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import time
import re
from urllib.parse import urljoin

# URL DE LA PAGINA A SCRAPEAR
BASE_URL = "https://books.toscrape.com/"
GOOGLE_BOOKS_API = "https://www.googleapis.com/books/v1/volumes"
# NOMBRE DE LA BASE DE DATOS
DB_NAME = 'libros.db'

# FUNCION PARA EXTRAER LAS CATEGORIAS DE LIBROS
def extraer_categorias():
    response = requests.get(BASE_URL)
    soup = BeautifulSoup(response.content, 'html.parser')
    categorias_html = soup.find('ul', class_='nav nav-list').find('ul').find_all('a')
    
    categorias = []
    
    for cat in categorias_html:
        nombre = cat.text.strip()
        url = BASE_URL + cat['href']
        categorias.append({'nombre': nombre, 'url': url})
    
    return categorias

# FUNCION PARA EXTRAER TODOS LOS LIBROS DE CADA CATEGORIA
def extraer_libros_de_categoria(url_categoria):
    libros_urls = []
    pagina_actual = url_categoria
    
    while pagina_actual:
        response = requests.get(pagina_actual)
        soup = BeautifulSoup(response.content, 'html.parser')
        libros = soup.find_all('article', class_='product_pod')

        for libro in libros:
            link = libro.find('h3').find('a')['href']

            if link.startswith('../../../'):
                link = link.replace('../../../', '')
            libro_url = urljoin(BASE_URL + 'catalogue/', link)
            libros_urls.append(libro_url)
        
        next_button = soup.find('li', class_='next')
        
        if next_button:
            next_url = next_button.find('a')['href']
            base_categoria = '/'.join(pagina_actual.split('/')[:-1])
            pagina_actual = f"{base_categoria}/{next_url}"
        else:
            pagina_actual = None
    
    return libros_urls

# FUNCION PARA EXTRAER STOCK
def extraer_stock(disponibilidad_texto):
    match = re.search(r'\((\d+) available\)', disponibilidad_texto)
    if match:
        return int(match.group(1))
    return 0

# FUNCION PARA EXTRAER DATOS DEL LIBRO
def extraer_libro(url_libro):
    response = requests.get(url_libro)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    titulo = soup.find('h1').text.strip()
    precio_texto = soup.find('p', class_='price_color').text.strip()
    precio = float(precio_texto.replace('£', ''))
    
    rating_class = soup.find('p', class_='star-rating')['class'][1]
    rating_map = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
    rating = rating_map.get(rating_class, 0)
    
    disponibilidad = soup.find('p', class_='instock availability').text.strip()
    stock = extraer_stock(disponibilidad)
    
    return {
        'titulo': titulo,
        'precio': precio,
        'rating': rating,
        'stock': stock
    }

def buscar_autores_google(titulo):
    try:
        params = {'q': f'intitle:{titulo}', 'maxResults': 1}
        response = requests.get(GOOGLE_BOOKS_API, params=params, timeout=5)
        data = response.json()
        
        if 'items' in data and len(data['items']) > 0:
            libro_info = data['items'][0]['volumeInfo']
            return libro_info.get('authors', ['Autor Desconocido'])
        else:
            return ['Autor Desconocido']
    except:
        return ['Autor Desconocido']




In [3]:
def insertar_categoria(cursor, nombre, url):
    cursor.execute('''
        INSERT OR IGNORE INTO categorias (nombre, url)
        VALUES (?, ?)
    ''', (nombre, url))
    
    cursor.execute('SELECT id FROM categorias WHERE nombre = ?', (nombre,))
    return cursor.fetchone()[0]

def insertar_autor(cursor, nombre):
    cursor.execute('''
        INSERT OR IGNORE INTO autores (nombre)
        VALUES (?)
    ''', (nombre,))
    
    cursor.execute('SELECT id FROM autores WHERE nombre = ?', (nombre,))
    return cursor.fetchone()[0]

def insertar_libro(cursor, libro, categoria_id):
    try:
        cursor.execute('''
            INSERT INTO libros (titulo, precio, rating, stock, categoria_id)
            VALUES (?, ?, ?, ?, ?)
        ''', (libro['titulo'], libro['precio'], libro['rating'], libro['stock'], categoria_id))
        return cursor.lastrowid
    except sqlite3.IntegrityError:
        cursor.execute('SELECT id FROM libros WHERE titulo = ? AND categoria_id = ?', (libro['titulo'], categoria_id))
        result = cursor.fetchone()
        return result[0] if result else None

def insertar_libro_autor(cursor, libro_id, autor_id):
    cursor.execute('''
        INSERT OR IGNORE INTO libros_autores (libro_id, autor_id)
        VALUES (?, ?)
    ''', (libro_id, autor_id))

In [4]:
def scrapear_todo():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    
    total_libros = 0
    total_categorias = 0
    
    try:
        categorias = extraer_categorias()
        
        for i, categoria in enumerate(categorias, 1):
            categoria_id = insertar_categoria(cursor, categoria['nombre'], categoria['url'])
            total_categorias += 1
            
            libros_urls = extraer_libros_de_categoria(categoria['url'])
            
            for j, url_libro in enumerate(libros_urls, 1):
                try:
                    libro = extraer_libro(url_libro)
                    libro_id = insertar_libro(cursor, libro, categoria_id)

                    autores = buscar_autores_google(libro['titulo'])
                    for autor_nombre in autores:
                        autor_id = insertar_autor(cursor, autor_nombre)
                        insertar_libro_autor(cursor, libro_id, autor_id)

                    total_libros += 1
                    time.sleep(0.2)

                except Exception as e:
                    continue
            
            conn.commit()
        
        print(f"\nScraping completado exitosamente.")
        print(f"Total de libros guardados: {total_libros}")
        print(f"Total de categorías procesadas: {total_categorias}")
        
    except KeyboardInterrupt:
        print("Scraping interrumpido por el usuario.")
        conn.commit()
    
    finally:
        conn.close()

scrapear_todo()


Scraping completado exitosamente.
Total de libros guardados: 1000
Total de categorías procesadas: 50


In [5]:
import time
import sqlite3

conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()

print("Consulta 1: Libros con 5 estrellas ordenados por precio (desc)")

inicio = time.time()

cursor.execute("""
    SELECT titulo, precio, rating
    FROM libros
    WHERE rating = 5
    ORDER BY precio DESC;
""")
resultados = cursor.fetchall()

fin = time.time()

for row in resultados:
    print(row)

print(f"Tiempo de ejecución: {fin - inicio:.4f} segundos")

Consulta 1: Libros con 5 estrellas ordenados por precio (desc)
('The Barefoot Contessa Cookbook', 59.92, 5)
('Life Without a Recipe', 59.04, 5)
('Approval Junkie: Adventures in Caring Too Much', 58.81, 5)
('How to Speak Golf: An Illustrated Guide to Links Lingo', 58.32, 5)
('Digital Fortress', 58.0, 5)
('The Sound Of Love', 57.84, 5)
('Travels with Charley: In Search of America', 57.82, 5)
('El Deafo', 57.62, 5)
('H is for Hawk', 57.42, 5)
('Immunity: How Elie Metchnikoff Changed the Course of Modern Medicine', 57.36, 5)
('The Disappearing Spoon: And Other True Tales of Madness, Love, and the History of the World from the Periodic Table of the Elements', 57.35, 5)
('Kitchens of the Great Midwest', 57.2, 5)
('A Piece of Sky, a Grain of Rice: A Memoir in Four Meditations', 56.76, 5)
('Into the Wild', 56.7, 5)
('Eleanor & Park', 56.51, 5)
('Abstract City', 56.37, 5)
('The False Prince (The Ascendance Trilogy #1)', 56.0, 5)
('Future Shock (Future Shock #1)', 55.65, 5)
("A New Earth: Awaken

In [6]:
conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()

inicio_indice = time.time()
cursor.execute("CREATE INDEX IF NOT EXISTS idx_libros_rating ON libros(rating)")
conn.commit()
fin_indice = time.time()

print(f"Tiempo en crear el índice: {fin_indice - inicio_indice:.4f} segundos")

print("Consulta 1: Libros con 5 estrellas ordenados por precio (desc)")
inicio_consulta = time.time()

cursor.execute("""
    SELECT titulo, precio, rating
    FROM libros
    WHERE rating = 5
    ORDER BY precio DESC;
""")
resultados = cursor.fetchall()

fin_consulta = time.time()

for row in resultados:
    print(row)

print(f"\nTiempo de ejecución de la consulta: {fin_consulta - inicio_consulta:.4f} segundos")


Tiempo en crear el índice: 0.0066 segundos
Consulta 1: Libros con 5 estrellas ordenados por precio (desc)
('The Barefoot Contessa Cookbook', 59.92, 5)
('Life Without a Recipe', 59.04, 5)
('Approval Junkie: Adventures in Caring Too Much', 58.81, 5)
('How to Speak Golf: An Illustrated Guide to Links Lingo', 58.32, 5)
('Digital Fortress', 58.0, 5)
('The Sound Of Love', 57.84, 5)
('Travels with Charley: In Search of America', 57.82, 5)
('El Deafo', 57.62, 5)
('H is for Hawk', 57.42, 5)
('Immunity: How Elie Metchnikoff Changed the Course of Modern Medicine', 57.36, 5)
('The Disappearing Spoon: And Other True Tales of Madness, Love, and the History of the World from the Periodic Table of the Elements', 57.35, 5)
('Kitchens of the Great Midwest', 57.2, 5)
('A Piece of Sky, a Grain of Rice: A Memoir in Four Meditations', 56.76, 5)
('Into the Wild', 56.7, 5)
('Eleanor & Park', 56.51, 5)
('Abstract City', 56.37, 5)
('The False Prince (The Ascendance Trilogy #1)', 56.0, 5)
('Future Shock (Future 

In [7]:
print("Consulta 2: Libros con más de 10 unidades en stock")

cursor.execute("""
    SELECT titulo, stock
    FROM libros
    WHERE stock > 10
""")
for row in cursor.fetchall():
    print(row)



Consulta 2: Libros con más de 10 unidades en stock
("It's Only the Himalayas", 19)
('Full Moon over Noah’s Ark: An Odyssey to Mount Ararat and Beyond', 15)
('See America: A Celebration of Our National Parks & Treasured Sites', 14)
('Sharp Objects', 20)
('In a Dark, Dark Wood', 18)
('The Past Never Ends', 16)
('A Murder in Time', 16)
('The Murder of Roger Ackroyd (Hercule Poirot #4)', 15)
('The Last Mile (Amos Decker #2)', 14)
('That Darkness (Gardiner and Renner #1)', 14)
('Tastes Like Fear (DI Marnie Rome #3)', 14)
('A Time of Torment (Charlie Parker #14)', 14)
('A Study in Scarlet (Sherlock Holmes #1)', 14)
('Poisonous (Max Revere Novels #3)', 12)
('Murder at the 42nd Street Library (Raymond Ambler #1)', 12)
('Most Wanted', 12)
('Hide Away (Eve Duncan #20)', 12)
('Boar Island (Anna Pigeon #19)', 12)
('The Widow', 11)
('Playing with Fire', 11)
('Tipping the Velvet', 20)
('Forever and Forever: The Courtship of Henry Longfellow and Fanny Appleton', 16)
('A Flight of Arrows (The Pathfind

In [8]:
print("Consulta 3: Libros ordenados por rating (mayor a menor)")

cursor.execute("""
    SELECT titulo, precio, rating
    FROM libros
    ORDER BY rating DESC;
""")
for row in cursor.fetchall():
    print(row)

Consulta 3: Libros ordenados por rating (mayor a menor)
('Dark Notes', 19.19, 5)
('The Art and Science of Low Carbohydrate Living', 52.98, 5)
('10-Day Green Smoothie Cleanse: Lose Up to 15 Pounds in 10 Days!', 49.71, 5)
('Suzie Snowflake: One beautiful flake (a self-esteem story)', 54.81, 5)
('(Un)Qualified: How God Uses Broken People to Do Big Things', 54.0, 5)
('All the Light We Cannot See', 29.87, 5)
('How to Stop Worrying and Start Living', 46.49, 5)
("A New Earth: Awakening to Your Life's Purpose", 55.65, 5)
("The Activist's Tao Te Ching: Ancient Advice for a Modern Revolution", 32.24, 5)
('The Four Agreements: A Practical Guide to Personal Freedom', 17.66, 5)
('Someone Like You (The Harrisons #2)', 52.79, 5)
('You (You #1)', 43.61, 5)
('The Elephant Tree', 23.82, 5)
('Born for This: How to Find the Work You Were Meant to Do', 21.59, 5)
('Made to Stick: Why Some Ideas Survive and Others Die', 38.85, 5)
('The Third Wave: An Entrepreneur’s Vision of the Future', 12.61, 5)
('Counted 

In [9]:
print("Consulta 5: Libro más barato por cada rating")

cursor.execute("""
    SELECT rating, titulo, precio
    FROM libros
    WHERE (rating, precio) IN (
        SELECT rating, MIN(precio)
        FROM libros
        GROUP BY rating
    )
    ORDER BY rating DESC;
""")
for row in cursor.fetchall():
    print(row)

Consulta 5: Libro más barato por cada rating
(5, 'An Abundance of Katherines', 10.0)
(4, 'The Origin of Species', 10.01)
(3, 'Patience', 10.16)
(2, 'The Tipping Point: How Little Things Can Make a Big Difference', 10.02)
(1, 'The Lucifer Effect: Understanding How Good People Turn Evil', 10.4)


In [10]:
conn.close()
print("Conexión cerrada")

Conexión cerrada
